In [2]:
import pandas as pd
import numpy as np
from darts.models import TFTModel
from pytorch_lightning.callbacks import EarlyStopping
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape
from darts.utils.timeseries_generation import datetime_attribute_timeseries

In [3]:
#import weather data Houston, CST/CDT, 2017-01-01 to 2022-12-30
weather_data = pd.read_csv('WeatherData/HOU.csv',
                           usecols=["valid","tmpf","dwpf","relh","p01i","sknt","feel"]) 

# rename columns    
weather_data.rename(columns={'valid': 'time','tmpf' : 'temp','dwpf' : 'dew_point','relh' : 'humidity', 'p01i' : 'precip','sknt' : 'wind_speed','feel' : 'feels_like' }, inplace=True)

# replace M (missing data) with Nan
weather_data.replace(to_replace='M', value=np.nan, inplace=True)

# replace T (trace precip) with 0.00
weather_data['precip'].replace(to_replace='T', value=0.00, inplace=True)

# # remove all rows containing Nan
weather_data.dropna(axis=0, how='any', inplace=True)

# convert values to float datatypes
weather_data = weather_data.astype({'temp':'float','dew_point':'float','humidity':'float','precip':'float','wind_speed':'float','feels_like':'float' })

# convert time to datetime datatype
weather_data['time'] = pd.to_datetime(weather_data['time'])

# resample weather dataframe by hour to match load dataset
weather_df = weather_data.resample('1H', on='time').mean()

# remove all rows containing Nan after resample
weather_df.dropna(axis=0, how='any', inplace=True)

# remove humidity and precipitation columns
weather_df.drop(columns=['humidity', 'precip'], axis=1, inplace=True)

In [4]:
# import load archive data
load_2017 = pd.read_excel('LoadData/Native_Load_2017.xlsx')
load_2017.rename(columns={'Hour Ending' : 'HourEnding'}, inplace=True)
load_2018 = pd.read_excel('LoadData/Native_Load_2018.xlsx')
load_2019 = pd.read_excel('LoadData/Native_Load_2019.xlsx')
load_2020 = pd.read_excel('LoadData/Native_Load_2020.xlsx')
load_2021 = pd.read_excel('LoadData/Native_Load_2021.xlsx')
load_2021.rename(columns={'Hour Ending' : 'HourEnding'}, inplace=True)
load_2022 = pd.read_excel('LoadData/Native_Load_2022.xlsx')
load_2022.rename(columns={'Hour Ending': "HourEnding"}, inplace=True)

dataframes = [load_2017,load_2018, load_2019, load_2020, load_2021, load_2022]

# concat files, include only load usage for Coastal area
load_test_data = pd.concat(dataframes, ignore_index=True)
load_test_data = load_test_data[['HourEnding', 'COAST']]

# change column names
load_test_data.rename(columns={'HourEnding' : 'time', 'COAST' : 'load'}, inplace=True)

# replace 2400 with 0:00 and convert to datetime type
load_test_data['time'] = load_test_data['time'].replace('24:00', '00:00', regex=True)
load_test_data['time'] = pd.to_datetime(load_test_data['time'])

load_df = load_test_data.resample('1H', on='time').mean()

# drop null values
load_df.dropna(axis=0, how='any', inplace=True)

In [5]:
# merge datasets
left = weather_df
right = load_df
full_df = pd.merge(left, right, on=['time'])

In [6]:
# date to split train/test sets
split_date = pd.Timestamp('20220101')

# timeseries of merged datasets
timeseries_df = TimeSeries.from_dataframe(full_df, freq='1H')

In [7]:
# create timeseries for load date
timeseries_target = TimeSeries.from_dataframe(load_df, fill_missing_dates=True, freq='1H', fillna_value=0.0)

# split load train/test datasets
train_target, test_target = timeseries_target.split_after(split_date)

# scale to training set
target_transformer = Scaler()
train_target_transformed = target_transformer.fit_transform(train_target)
test_target_transformed = target_transformer.transform(test_target)
timeseries_target_transformed = target_transformer.transform(timeseries_target)



In [8]:

# create timeseries for weather features
timeseries_features = TimeSeries.from_dataframe(weather_df, fill_missing_dates=True, freq='1H', fillna_value=0.0)

# split train/test weather sets
train_features, test_features = timeseries_features.split_after(split_date)

# scale to training set
feature_transformer = Scaler()
train_features_transformed = feature_transformer.fit_transform(train_features)
test_features_transformed = feature_transformer.transform(test_features)
timeseries_features_transformed = feature_transformer.transform(timeseries_features)

# past covariates 
past_covariates = train_features_transformed

In [9]:
# time covariates creation
time_cov = datetime_attribute_timeseries(timeseries_target.time_index, attribute='hour')
time_cov = time_cov.stack(datetime_attribute_timeseries(time_cov.time_index, attribute='day_of_week'))
time_cov = time_cov.stack(datetime_attribute_timeseries(time_cov.time_index, attribute='month'))
time_cov = time_cov.stack(datetime_attribute_timeseries(time_cov.time_index, attribute='year'))

time_cov = time_cov.add_holidays(country_code='US')

# convert to dataframe to add other features
time_cov_df = time_cov.pd_dataframe()

# add seasons
time_cov_df['is_summer'] = np.where((time_cov_df.index.month >= 6) & (time_cov_df.index.month <= 8),1,0)       # summer months 6-8
time_cov_df['is_winter'] = np.where((time_cov_df.index.month == 12) | (time_cov_df.index.month <= 2), 1, 0)    # winter months 12-2
time_cov_df['is_spring'] = np.where((time_cov_df.index.month >= 3) & (time_cov_df.index.month <= 5), 1, 0)     # spring months 3-5
time_cov_df['is_autumn'] = np.where((time_cov_df.index.month >= 9) & (time_cov_df.index.month <= 11), 1, 0)    # autumn months 9-11

# add weekends
time_cov_df['is_weekend'] = np.where((time_cov_df.index.weekday >= 5), 1, 0) 

# back to Timeseries
time_cov = TimeSeries.from_dataframe(time_cov_df)

# split time feature train/test sets
time_cov_train, time_cov_test = time_cov.split_after(split_date)

# scale time covariates 
time_transformer = Scaler()
time_cov_train_transformed = time_transformer.fit_transform(time_cov_train)
time_cov_test_transformed = time_transformer.transform(time_cov_test)
time_cov_transformed = time_transformer.transform(time_cov)

# define future covariates
future_covariates = time_cov_transformed # time features

In [10]:
# model
model_name = 'forecast_model'

#hyperparamters
LOAD=False
EPOCHS=2        #training cycles
INLEN=24         #number of inputs/columns
OUTLEN=24        #forecast periods/output size
HIDDEN=32        #hidden layer
LSTMLAYERS=1    
ATTH=1          #attention heads
DROPOUT=0.1     #dropout rate
BATCH=64        #batch size  
RAND=40         #random seed
N_SAMPLES=100   #prediction samples

In [11]:
early_stop = EarlyStopping(
    monitor='train_loss',
    patience=5,
    min_delta=0.05,
    mode='min'
)

pl_trainer_kwargs = {'callbacks': early_stop}

In [12]:
# model setup with hyperparameterrs
forecast_model = TFTModel(
    input_chunk_length=INLEN,
    output_chunk_length=OUTLEN,
    hidden_size=HIDDEN,
    lstm_layers=LSTMLAYERS,
    num_attention_heads=ATTH,
    dropout=DROPOUT,
    batch_size=BATCH,
    n_epochs=EPOCHS,
    random_state=RAND,
    add_encoders={
        "cyclic": {"future": ["hour", "dayofweek", "month"]},
        'transformer': Scaler()
    },
    optimizer_kwargs={"lr": 1e-3}
)

In [ ]:
# train model
forecast_model.fit(train_target_transformed,verbose=True)

In [ ]:
path = F"Models\{model_name}"
forecast_model.save(path)

In [13]:
forecast_model = TFTModel.load('Models\\forecast_model')

In [14]:
def model_eval(model, n, test_series):
    pred_series = model.predict(n=n, num_samples=N_SAMPLES)
    print('MAPE: {: .2f}%'.format(mape(test_series,pred_series)))


In [16]:
model_eval(forecast_model, 12, test_target_transformed)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 11.27it/s]
MAPE:  4.54%
